In [1]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm

In [2]:
# Đường dẫn gốc
dataset_path = 'd:/MRI_4/dataset_2'
output_path = 'd:/MRI_4/dataset_2_reorganized'

In [3]:
# Tạo cấu trúc thư mục mới
folders = ['train', 'validation', 'test']
classes = ['Glioma', 'Meningioma', 'No_Tumor', 'Pituitary']

In [4]:
# Tạo thư mục mới
for folder in folders:
    folder_path = os.path.join(output_path, folder)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    for class_name in classes:
        class_path = os.path.join(folder_path, class_name)
        if not os.path.exists(class_path):
            os.makedirs(class_path)


In [5]:
# Kiểm tra cấu trúc thư mục gốc
def check_dataset_structure():
    print("=== KIỂM TRA CẤU TRÚC DATASET 2 ===")
    
    # Kiểm tra thư mục Training
    print("\nThư mục Training:")
    training_folders = os.listdir(os.path.join(dataset_path, 'Training'))
    print(f"Các thư mục con: {training_folders}")
    
    for folder in training_folders:
        path = os.path.join(dataset_path, 'Training', folder)
        if os.path.isdir(path):
            files = os.listdir(path)
            print(f"  - {folder}: {len(files)} files")
            
            # Kiểm tra xem có thư mục con không
            subdirs = [f for f in files if os.path.isdir(os.path.join(path, f))]
            if subdirs:
                print(f"    Có {len(subdirs)} thư mục con: {subdirs}")
    
    # Kiểm tra thư mục Testing
    print("\nThư mục Testing:")
    testing_folders = os.listdir(os.path.join(dataset_path, 'Testing'))
    print(f"Các thư mục con: {testing_folders}")
    
    for folder in testing_folders:
        path = os.path.join(dataset_path, 'Testing', folder)
        if os.path.isdir(path):
            files = os.listdir(path)
            print(f"  - {folder}: {len(files)} files")
            
            # Kiểm tra xem có thư mục con không
            subdirs = [f for f in files if os.path.isdir(os.path.join(path, f))]
            if subdirs:
                print(f"    Có {len(subdirs)} thư mục con: {subdirs}")

# Chạy hàm kiểm tra
check_dataset_structure()

=== KIỂM TRA CẤU TRÚC DATASET 2 ===

Thư mục Training:
Các thư mục con: ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
  - glioma_tumor: 826 files
  - meningioma_tumor: 822 files
  - no_tumor: 395 files
  - pituitary_tumor: 827 files

Thư mục Testing:
Các thư mục con: ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
  - glioma_tumor: 100 files
  - meningioma_tumor: 115 files
  - no_tumor: 105 files
  - pituitary_tumor: 74 files


In [6]:
def reorganize_dataset():
    # Biến đổi tên thư mục để phù hợp với cấu trúc
    class_mapping = {
        # Mapping cho tập Training
        'glioma_tumor': 'Glioma',
        'meningioma_tumor': 'Meningioma', 
        'no_tumor': 'No_Tumor',
        'pituitary_tumor': 'Pituitary',
        
        # Mapping cho tập Testing
        'glioma': 'Glioma',
        'meningioma': 'Meningioma',
        'notumor': 'No_Tumor',
        'pituitary': 'Pituitary'
    }
    
    # Thu thập tất cả các file
    all_files = {}
    
    # Thu thập file từ tập Training
    for class_folder in os.listdir(os.path.join(dataset_path, 'Training')):
        standard_class_name = class_mapping.get(class_folder)
        if standard_class_name:
            class_path = os.path.join(dataset_path, 'Training', class_folder)
            if standard_class_name not in all_files:
                all_files[standard_class_name] = []
            
            for img_file in os.listdir(class_path):
                all_files[standard_class_name].append(os.path.join(class_path, img_file))
    
    # Thu thập file từ tập Testing
    for class_folder in os.listdir(os.path.join(dataset_path, 'Testing')):
        standard_class_name = class_mapping.get(class_folder)  # Sử dụng mapping mới
        if standard_class_name:
            class_path = os.path.join(dataset_path, 'Testing', class_folder)
            if standard_class_name not in all_files:
                all_files[standard_class_name] = []
                
            for img_file in os.listdir(class_path):
                all_files[standard_class_name].append(os.path.join(class_path, img_file))
    
    # Kiểm tra tổng số ảnh trước khi chia
    total_images = sum(len(files) for files in all_files.values())
    print(f"Tổng số ảnh đã thu thập: {total_images}")
    
    # Phần còn lại giữ nguyên
    for class_name, files in all_files.items():
        random.shuffle(files)
        
        # Chia tập train và temp (temp bao gồm validation và test)
        train_files, temp_files = train_test_split(files, train_size=0.7, random_state=42)
        
        # Chia temp thành validation và test (50-50)
        validation_files, test_files = train_test_split(temp_files, train_size=0.5, random_state=42)
        
        print(f"Class {class_name}: {len(train_files)} train, {len(validation_files)} validation, {len(test_files)} test")
        
        # Sao chép file vào thư mục mới tương ứng
        for src_file in tqdm(train_files, desc=f"Copying {class_name} train files"):
            dst_file = os.path.join(output_path, 'train', class_name, os.path.basename(src_file))
            shutil.copy2(src_file, dst_file)
            
        for src_file in tqdm(validation_files, desc=f"Copying {class_name} validation files"):
            dst_file = os.path.join(output_path, 'validation', class_name, os.path.basename(src_file))
            shutil.copy2(src_file, dst_file)
            
        for src_file in tqdm(test_files, desc=f"Copying {class_name} test files"):
            dst_file = os.path.join(output_path, 'test', class_name, os.path.basename(src_file))
            shutil.copy2(src_file, dst_file)

In [7]:
# Thực thi việc tái tổ chức dữ liệu
reorganize_dataset()

Tổng số ảnh đã thu thập: 3264
Class Glioma: 648 train, 139 validation, 139 test


Copying Glioma test files: 100%|██████████| 139/139 [00:01<00:00, 71.73it/s]


Class Meningioma: 655 train, 141 validation, 141 test


Copying Meningioma test files: 100%|██████████| 141/141 [00:02<00:00, 69.75it/s]


Class No_Tumor: 350 train, 75 validation, 75 test


Copying No_Tumor test files: 100%|██████████| 75/75 [00:00<00:00, 88.24it/s]


Class Pituitary: 630 train, 135 validation, 136 test


Copying Pituitary test files: 100%|██████████| 136/136 [00:01<00:00, 87.52it/s]


In [8]:
# Kiểm tra số lượng ảnh trong từng tập
for folder in folders:
    print(f"\n{folder.upper()} SET:")
    for class_name in classes:
        class_path = os.path.join(output_path, folder, class_name)
        num_images = len(os.listdir(class_path))
        print(f"  {class_name.capitalize()}: {num_images} images")


TRAIN SET:
  Glioma: 648 images
  Meningioma: 655 images
  No_tumor: 298 images
  Pituitary: 630 images

VALIDATION SET:
  Glioma: 139 images
  Meningioma: 141 images
  No_tumor: 69 images
  Pituitary: 135 images

TEST SET:
  Glioma: 139 images
  Meningioma: 141 images
  No_tumor: 73 images
  Pituitary: 136 images
